# Module 2: Pathfinding & Graph Search 

Graph search algorithms explore a graph either for general discovery or explicit search. These algorithms carve paths through the graph, but there is no expectation that those paths are computationally optimal.

Pathfinding algorithms build on top of graph search algorithms and explore routes between nodes, starting at one node and traversing through relationships until the sdestination has been reached.

In this notebook we'll learn how to use these algorithms in Spark and Neo4j. Before we get started let's import those libraries:

In [67]:
from pyspark.sql.types import *
from graphframes import *
from neo4j import GraphDatabase
import pandas as pd
from pyspark.sql import functions as F

## The Transport Graph

The transport graph is a subset of the data from the [International E-road Network](elbruz.org/e-roads).

### Importing the Data into Apache Spark

First let's import the data into Apache Spark. The function below creates a GraphFrame based on these CSV files:

In [50]:
def create_transport_graph():
    node_fields = [
        StructField("id", StringType(), True),
        StructField("latitude", FloatType(), True),
        StructField("longitude", FloatType(), True),
        StructField("population", IntegerType(), True)
    ]
    nodes = spark.read.csv("data/transport-nodes.csv", header=True,
                           schema = StructType(node_fields))
    
    rels = spark.read.csv("data/transport-relationships.csv", header=True)
    reversed_rels = (rels.withColumn("newSrc", rels.dst)
                     .withColumn("newDst", rels.src)
                     .drop("dst", "src")
                     .withColumnRenamed("newSrc", "src")
                     .withColumnRenamed("newDst", "dst")
                     .select("src", "dst", "relationship", "cost"))
    relationships = rels.union(reversed_rels)
    
    return GraphFrame(nodes, relationships)

And now let's call that function, and assign the GraphFrame to a variable:

In [51]:
g = create_transport_graph()

### Importing the Data into Neo4j

Now we'll import the data into Neo4j. 

First let's create a connection to the database. We'll need to update the password to use the one that we chose when creating our database:

In [9]:
user = "neo4j"
password = "neo"
driver = GraphDatabase.driver("bolt://localhost", auth=(user, password))

And now we can create nodes and relationships based on the data in the CSV files:

In [49]:
with driver.session() as session:
    session.run("""
    WITH "https://github.com/neo4j-graph-analytics/book/raw/master/data/" AS base
    WITH base + "transport-nodes.csv" AS uri
    LOAD CSV WITH HEADERS FROM uri AS row
    MERGE (place:Place {id:row.id})
    SET place.latitude = toFloat(row.latitude),
        place.longitude = toFloat(row.latitude),
        place.population = toInteger(row.population)
    """)
    
    session.run("""
    WITH "https://github.com/neo4j-graph-analytics/book/raw/master/data/" AS base
    WITH base + "transport-relationships.csv" AS uri
    LOAD CSV WITH HEADERS FROM uri AS row
    MATCH (origin:Place {id: row.src})
    MATCH (destination:Place {id: row.dst})
    MERGE (origin)-[:EROAD {distance: toInteger(row.cost)}]->(destination)
    """)

## Breadth First Search

Spark’s implementation of the Breadth First Search algorithm finds the shortest path between two nodes by the number of relationships (i.e., hops) between them. You can explicitly name your target node or add criteria to be met.

The diagram below shows the order in which we would visit the nodes of our transport graph if we were performing a breadth first search that started from the Dutch city, Den Haag (in English, The Hague). The numbers next to the city name indicate the order in which each node is visited.

![](images/traversing_bfs.svg)

For example, we can use the bfs function to find the first medium-sized (by European standards) city that has a population of between 100,000 and 300,000 people.

Let’s first check which places have a population matching those criteria:

In [52]:
(g.vertices
 .filter("population > 100000 and population < 300000")
 .sort("population")
 .show())

+----------+--------+---------+----------+
|        id|latitude|longitude|population|
+----------+--------+---------+----------+
|Colchester|51.88921|  0.90421|    104390|
|   Ipswich|52.05917|  1.15545|    133384|
+----------+--------+---------+----------+



There are only two places matching our criteria, and we’d expect to reach Ipswich first based on a breadth first search.

The following code finds the shortest path from Den Haag to a medium-sized city:

In [73]:
from_expr = "id='Den Haag'"
to_expr = "population > 100000 and population < 300000 and id <> 'Den Haag'"
result = g.bfs(from_expr, to_expr)
result.show(truncate=False)

+---------------------------------------+---------------------------------------+------------------------------------------+------------------------------------------+-------------------------------------+--------------------------------+------------------------------------+
|from                                   |e0                                     |v1                                        |e1                                        |v2                                   |e2                              |to                                  |
+---------------------------------------+---------------------------------------+------------------------------------------+------------------------------------------+-------------------------------------+--------------------------------+------------------------------------+
|[Den Haag, 52.078663, 4.288788, 514861]|[Den Haag, Hoek van Holland, EROAD, 27]|[Hoek van Holland, 51.9775, 4.13333, 9382]|[Hoek van Holland, Felixstowe, EROAD, 207]|[Feli

By default the result has fields for nodes and relationships. We're only interested in the nodes, so let's remove the relationship columns. All those columns start with 'e', so the following code will remove them:

In [72]:
columns = [F.col(column + ".id").alias(column) 
           for column in result.columns 
           if not column.startswith("e")]

result.select(columns).show()

+--------+----------------+----------+-------+
|    from|              v1|        v2|     to|
+--------+----------------+----------+-------+
|Den Haag|Hoek van Holland|Felixstowe|Ipswich|
+--------+----------------+----------+-------+



The image below shows the route that the breadth first search took to get from Den Haag to Ipswich:

![](images/traversing_bfs_highlighted.svg)

Colchester was the only other node that satisfied our criteria but, as we can see, it comes after Ipswich in the breadth first search.

## Shortest Path


The Shortest Path algorithm calculates the shortest (weighted) path between a pair of nodes. It’s useful for user interactions and dynamic workflows because it works in real time.

<img src="images/Edsger_Wybe_Dijkstra.jpg" alt="drawing" width="120" style="float:right"/>

Dijkstra’s Shortest Path algorithm operates by first finding the lowest-weight relationship from the start node to directly connected nodes. It keeps track of those weights and moves to the “closest” node. 

It then performs the same calculation, but now as a cumulative total from the start node. The algorithm continues to do this, evaluating a “wave” of cumulative weights and always choosing the lowest weighted cumulative path to advance along, until it reaches the destination node.

We're going to explore how to use this one using Neo4j. We'll start by computing unweighted shortest paths, which we can do by passing `null` as the 3rd parameter to the procedure:

In [75]:
query = """
MATCH (source:Place {id: $source}),
      (destination:Place {id: $destination})
CALL algo.shortestPath.stream(source, destination, null)
YIELD nodeId, cost
RETURN algo.getNodeById(nodeId).id AS place, cost
"""

params = {
    "source": "Amsterdam",
    "destination": "London"
}

with driver.session() as session:
    rows = session.run(query, params)
    df = pd.DataFrame([dict(record) for record in rows])

display(df)

,cost,place
0,0.0,Amsterdam
1,1.0,Immingham
2,2.0,Doncaster
3,3.0,London


The cost here is the cumulative number of relationships. If we want to find the cost in terms of distance, we can write some post processing code in Cypher:

In [80]:
query = """
MATCH (source:Place {id: $source}),
      (destination:Place {id: $destination})
CALL algo.shortestPath.stream(source, destination, null)
YIELD nodeId, cost

// collect all the nodes in the path
WITH collect(algo.getNodeById(nodeId)) AS path

// iterate over pairs of nodes, 
// find the EROAD relationship between them, 
// and extract the distance
UNWIND range(0, size(path)-1) AS index
WITH path[index] AS current, path[index+1] AS next
WITH current, next, [(current)-[r:EROAD]-(next) | r.distance][0] AS distance

// create a list of places and the distance of that part of the journey
WITH collect({current: current, next:next, distance: distance}) AS stops

// iterate over that list, computing the cumulative distance 
// at each stage of the journey
UNWIND range(0, size(stops)-1) AS index
WITH stops[index] AS location, stops, index
RETURN location.current.id AS place,
reduce(acc=0.0,
       distance in [stop in stops[0..index] | stop.distance] |
       acc + distance) AS cost
"""

params = {
    "source": "Amsterdam",
    "destination": "London"
}

with driver.session() as session:
    rows = session.run(query, params)
    df = pd.DataFrame([dict(record) for record in rows])

display(df)

,cost,place
0,0.0,Amsterdam
1,369.0,Immingham
2,443.0,Doncaster
3,720.0,London


The unweighted shortest path from Amsterdam to London, routing us through the fewest number of cities. It has a total cost of 720 km.

![](images/unweighted_shortest.svg)

Now let's have a look at how to find the shortest path while taking into account the distance of each road.

## Weighted Shortest Path

To do this we need to pass the name of the property that contains the relationship weight. 

In [26]:
query = """
MATCH (source:Place {id: $source}),
      (destination:Place {id: $destination})
CALL algo.shortestPath.stream(source, destination, "distance")
YIELD nodeId, cost
RETURN algo.getNodeById(nodeId).id AS place, cost
"""

params = {
    "source": "Amsterdam",
    "destination": "London"
}

with driver.session() as session:
    rows = session.run(query, params)
    df = pd.DataFrame([dict(record) for record in rows])

display(df)

,cost,place
0,0.0,Amsterdam
1,59.0,Den Haag
2,86.0,Hoek van Holland
3,293.0,Felixstowe
4,315.0,Ipswich
5,347.0,Colchester
6,453.0,London


## All Pairs Shortest Path

In [27]:
result = g.shortestPaths(["Colchester", "Immingham", "Hoek van Holland"])
result.sort(["id"]).select("id", "distances").show(truncate=False)

+----------------+--------------------------------------------------------+
|id              |distances                                               |
+----------------+--------------------------------------------------------+
|Amsterdam       |[Immingham -> 1, Hoek van Holland -> 2, Colchester -> 4]|
|Colchester      |[Colchester -> 0, Immingham -> 3, Hoek van Holland -> 3]|
|Den Haag        |[Hoek van Holland -> 1, Immingham -> 2, Colchester -> 4]|
|Doncaster       |[Immingham -> 1, Colchester -> 2, Hoek van Holland -> 4]|
|Felixstowe      |[Hoek van Holland -> 1, Colchester -> 2, Immingham -> 4]|
|Gouda           |[Hoek van Holland -> 2, Immingham -> 3, Colchester -> 5]|
|Hoek van Holland|[Hoek van Holland -> 0, Immingham -> 3, Colchester -> 3]|
|Immingham       |[Immingham -> 0, Colchester -> 3, Hoek van Holland -> 3]|
|Ipswich         |[Colchester -> 1, Hoek van Holland -> 2, Immingham -> 4]|
|London          |[Colchester -> 1, Immingham -> 2, Hoek van Holland -> 4]|
|Rotterdam  

In [28]:
query = """
CALL algo.allShortestPaths.stream(null)
YIELD sourceNodeId, targetNodeId, distance
WHERE sourceNodeId < targetNodeId
RETURN algo.getNodeById(sourceNodeId).id AS source,
algo.getNodeById(targetNodeId).id AS target,
distance
ORDER BY distance DESC
LIMIT 10
"""

with driver.session() as session:
    rows = session.run(query)
    df = pd.DataFrame([dict(record) for record in rows])

display(df)

,distance,source,target
0,5.0,London,Gouda
1,5.0,Utrecht,Ipswich
2,5.0,London,Rotterdam
3,5.0,Colchester,Gouda
4,5.0,Utrecht,Colchester
5,4.0,Amsterdam,Colchester
6,4.0,Immingham,Ipswich
7,4.0,Den Haag,Colchester
8,4.0,Doncaster,Felixstowe
9,4.0,Utrecht,Felixstowe


In [29]:
query = """
CALL algo.allShortestPaths.stream("distance")
YIELD sourceNodeId, targetNodeId, distance
WHERE sourceNodeId < targetNodeId
RETURN algo.getNodeById(sourceNodeId).id AS source,
algo.getNodeById(targetNodeId).id AS target,
distance
ORDER BY distance DESC
LIMIT 10
"""

with driver.session() as session:
    rows = session.run(query)
    df = pd.DataFrame([dict(record) for record in rows])

display(df)

,distance,source,target
0,529.0,Doncaster,Hoek van Holland
1,528.0,Doncaster,Rotterdam
2,524.0,Doncaster,Gouda
3,511.0,Immingham,Felixstowe
4,502.0,Den Haag,Doncaster
5,489.0,Immingham,Ipswich
6,489.0,Utrecht,Doncaster
7,460.0,Utrecht,London
8,457.0,Immingham,Colchester
9,455.0,Immingham,Hoek van Holland


## Single Source Shortest Path

In [33]:
query = """
MATCH (n:Place {id:$place})
CALL algo.shortestPath.deltaStepping.stream(n, "distance", 1.0)
YIELD nodeId, distance
WHERE algo.isFinite(distance)
RETURN algo.getNodeById(nodeId).id AS destination, distance
ORDER BY distance
"""

params = {
    "place": "London"
}

with driver.session() as session:
    rows = session.run(query, params)
    df = pd.DataFrame([dict(record) for record in rows])

display(df)

,destination,distance
0,London,0.0
1,Colchester,106.0
2,Ipswich,138.0
3,Felixstowe,160.0
4,Doncaster,277.0
5,Immingham,351.0
6,Hoek van Holland,367.0
7,Den Haag,394.0
8,Rotterdam,400.0
9,Gouda,425.0


## Minimum Spanning Tree

In [35]:
query = """
MATCH (n:Place {id:$place})
CALL algo.spanningTree.minimum("Place", "EROAD", "distance", id(n),
{write:true, writeProperty:"MINST"})
YIELD loadMillis, computeMillis, writeMillis, effectiveNodeCount
RETURN loadMillis, computeMillis, writeMillis, effectiveNodeCount
"""

params = {
    "place": "Amsterdam"
}

with driver.session() as session:
    rows = session.run(query, params)
    df = pd.DataFrame([dict(record) for record in rows])

display(df)

,computeMillis,effectiveNodeCount,loadMillis,writeMillis
0,8,12,32,15


In [46]:
query = """
MATCH path = (n:Place {id:$place})-[:MINST*]->(end)
WHERE not((end)-[:MINST]->())
WITH relationships(path) AS rels
UNWIND rels AS rel
RETURN startNode(rel).id AS source, endNode(rel).id AS destination, rel.distance AS cost
"""

params = {
    "place": "Amsterdam"
}

with driver.session() as session:
    rows = session.run(query, params)
    df = pd.DataFrame([dict(record) for record in rows])

display(df)

,cost,destination,source
0,46.0,Utrecht,Amsterdam
1,35.0,Gouda,Utrecht
2,25.0,Rotterdam,Gouda
3,26.0,Den Haag,Rotterdam
4,27.0,Hoek van Holland,Den Haag
5,207.0,Felixstowe,Hoek van Holland
6,22.0,Ipswich,Felixstowe
7,32.0,Colchester,Ipswich
8,106.0,London,Colchester
9,277.0,Doncaster,London
